<a href="https://colab.research.google.com/github/opherdonchin/ModelsOfTheMotorSystems2021-2/blob/master/In%20class%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple structural model of the muscle spindle

In his 1983 [Journal of Neurophysiology paper](https://journals.physiology.org/doi/abs/10.1152/jn.1983.49.4.989?rfr_dat=cr_pub++0pubmed&url_ver=Z39.88-2003&rfr_id=ori%3Arid%3Acrossref.org), A model of spindle afferent response to muscle stretch, Hasan puts forward what is called a "structural model" of muscle spindles. The model is called structural because it has multiple components that reflect actual parts of the biological system.

The model has essentially two components: the sensory element of the muscle spindle $z$ and the non-sensory element $y$. Together they add up to the total muscle length $x$. The firing rate of the muscle spindle is a function of $z$ and $\frac{dz}{dt}$ and is given by the formula ($H$ and $p$ constants):

$$
r(t) = H\left( z(t) + p \frac{dz(t)}{dt} \right)
$$

The length and rate of change of the sensory component is related to the length and rate of change of the muscle itself through the physical coupling of the lengths:

$$
x = z + y
$$

and the tension between the two components ($E$, $C$, $A$ and $k$ constants)

$$
f(t) = 
E(y-C)\left(1+\left(\frac{1}{A}\frac{dy}{dt}\right)^{\frac{1}{3}}\right) =
kz
$$

In other words, the tension in the muscle rises slowly with velocity and this causes increaed stretching of the sensory component when the muscle lengthens.

In this exercise, we will take code which I've written to implement this model and observe how it behaves in different conditions.

# The model implementation
The function `dz_from_state` implements the differential equation that relates $x$, $\frac{dx}{dt}$, and $z$ to $\frac{dz}{dt}$. We will later use this in the differential equation solver. A little bit of algebra has been applied to the equations above to find $\frac{dz}{dt}$ (called `dz`) in terms of the other variables.

The function `fr_from_state` implements the translation from the state of the sensory component to the firing rate.

In [1]:
from scipy.integrate import odeint, solve_ivp
import numpy as np
import matplotlib.pyplot as plt

A = 0.3
B = 250
C = -15
def dz_from_state(t, z, x, dx):
  y = x(t)-z
  delta_y = y-C
  dz = dx(t) - A*(((B-1)*z - delta_y)/delta_y)**3
  return dz

H = 350
P = 0.1
def fr_from_state(z, dz):
  fr = H*(z + P*dz)
  return fr



# Manipulating the muscle length and deriving firing rate
This code takes muscle length, solves the differential equation and generates the appropriate firing rate.

You can change the function of muscle length and change accordingly to try different options.

You can make this code better by taking the numerical derivative of `x` to get `dx`. Otherwise, it is on the programmer to keep them consistent and there could be meaningless results.

In [ ]:


t_span = (0.0, 10.0)
t_eval = np.arange(t_span[0], t_span[1], 0.005)


x = lambda t : 15
dx = lambda t : 0

z0 = np.atleast_1d( (x(0)-C) / B )

result = solve_ivp(dz_from_state, t_span, z0, args=(x, dx), method='LSODA', t_eval=t_eval)

t = result.t
z = result.y[0,:]
dz = dz_from_state(t, z, x, dx)

fr = fr_from_state(z, dz)


# Plotting
This plots the firing rate as a function of time.

You can make this code better by adding the muscle length on this plot or below it.

In [ ]:
fig , ax = plt.subplots(figsize=(9 ,8))
plt.title('Firing rate of spindle' , fontsize = 24)
plt.ylabel('fr(sp/sec)' , fontsize = 21)
plt.xlabel('t(sec)', fontsize = 21)
plt.plot(t, fr, linestyle='-', linewidth=2.0, color='blue')
plt.show()

Please check the following:

1. For constant muscle length, do we see a linear relationship between muscle length and firing rate?
2. Show that when a ramp and hold is applied to the muscle, this leads to:
  - strong transients when the ramp is started and stopped
  - ramping increase in firing rate during the ramp
  - constant firing rate that depends on final length at the end of the ramp
3. Later models have an explicit way of modeling the fusiform input. For this model, fusiform input can only be modeled by changes in parameters. Remember that $\gamma$-dynamic inputs should increase the sensitivity to velocity and $\gamma$-static should increase the sensitivity to length. You can check this. The authors suggest:
  - Activation of the $\gamma$-dynamic inputs: `A = 0.1`, `B = 125`, `C = -15`, `h = 250`, and `p = 0.1`
  - Activation of the $\gamma$-static inputs: `A = 100`, `B = 100`, `C = -25`, `h = 200`, and `p = 0.1`